# Analisis Salarios Colombia

por: Jose R. Zapata - https://joserzapata.github.io/

Link: https://joserzapata.github.io/post/


En base a los cambios que se han presentado en la reforma tributaria en Colombia en el 2022 y como se ven afectados los salarios de los trabajadores, se realiza un análisis de los salarios en Colombia para entender cuanto se le descuenta a un trabajador y cuanto seria el dinero que recibe en su bolsillo.


## Descuentos en salarios en Colombia

Para conocer cuanto se le descuenta a un trabajador en Colombia es necesario  tener en cuenta los decuentos por:
 - Salud (4%)
 - Pension (4%)
 - ARL (0.5% pero varia segun el riesgo)
 - Retencion en la fuente [Articulo 383 del estatuto tributario](https://estatuto.co/383)

Los calculos son diferentes para asalariados que reciben el [salario integral](https://loggro.com/blog/articulo/como-calcular-el-salario-integral-loggro-nomina/)

la tabla de retención en la fuente que esta en base a los valores de UVT (Unidad de Valor Tributario) y que se actualiza cada año. 

### Porcentaje de retencion en la fuente

los rango estan representados en UVT (2023) = $42.412 pesos

 | Rangos en UVT  | Tarifa marginal | retención en la fuente |
| -------------   | --------------- | ----------------------- |
| 0 a 95          | 0%  |                                     |
| \>95 a 150      | 19% | (Ingreso laboral gravado expresado en UVT menos 95 UVT)\*19%               |
| \>150 a 360     | 28% | (Ingreso laboral gravado expresado en UVT menos 150 UVT)\*28% más 10 UVT |
| \>360 a  640    | 33% | (Ingreso laboral gravado expresado en UVT menos 360 UVT)\*33% más 69 UVT   |
| \>640 a  945    | 35% | (Ingreso laboral gravado expresado en UVT menos 640 UVT)\*35% más 162 UVT  |
| \>945 a  2300   | 37% | (Ingreso laboral gravado expresado en UVT menos 945 UVT)\*37% más 268 UVT  |
| \>2300 En adelante |  39%| (Ingreso laboral gravado expresado en UVT menos 2300 UVT)\*39% más 770 UVT |

Nota: Sera unicamente para salarios y no tendra en cuenta ingresos extra. Ademas el analisis se va realizar para un rago salarial de 1'160.000 pesos (salario minimo) a 50'894.400 pesos (1200 UVT)




[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JoseRZapata/covid_plots/blob/main/notebooks/Covid19_Visualizacion_es.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/JoseRZapata/covid_plots/HEAD?labpath=notebooks%2FCovid19_Visualizacion_es.ipynb)  [![nbviewer](https://img.shields.io/badge/render-nbviewer-orange.svg)](https://nbviewer.jupyter.org/github/JoseRZapata/covid_plots/blob/main/notebooks/Covid19_Visualizacion_es.ipynb)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

#import chart_studio

import plotly.express as px

In [2]:
#print pandas, px an numpy version
print('pandas version: ', pd.__version__)
print('numpy version: ', np.__version__)

pandas version:  2.1.3
numpy version:  1.26.2


In [3]:
# Leer archivo con los datos de acceso a chart_studio
# si no esta, dejar los campos vacios
#try:
#    with open("../../info_chart.csv","r") as f:
#        info_user = f.read().split(";")
#        username = info_user[0] # your username
#        api_key = info_user[1] # your api key
#        print('api_key loaded!')
#except:
#    username = '' # your username
#    api_key =''
#
#if api_key: chart_studio.tools.set_credentials_file(username=username, api_key=api_key)


# Codigo Fuente Jupyter notebook
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JoseRZapata/covid_plots/blob/main/notebooks/Covid19_Visualizacion_es.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/JoseRZapata/covid_plots/HEAD?labpath=notebooks%2FCovid19_Visualizacion_es.ipynb)  [![nbviewer](https://img.shields.io/badge/render-nbviewer-orange.svg)](https://nbviewer.jupyter.org/github/JoseRZapata/covid_plots/blob/main/notebooks/Covid19_Visualizacion_es.ipynb)

In [4]:
# constantes globales
uvt = 42_412
salario_minimo = 1_160_000
subsido_transporte = 140_606
# salario integral = 10 salarios minimos + 30% prestaciones
salario_integral = salario_minimo*(10)*(1 + 0.3)
print(f'salario integral: {salario_integral}')

salario integral: 15080000.0


En base a la tabla de rangos de retencion en la fuente presentado anteriormente,
voy a generar los rangos de ingresos mensuales.

Lo valores empezaran desde el salario minimo legal vigente en Colombia para el 2023 que es de 1'160.000 pesos colombianos.
Equivalente a 27.351 UVT (Unidad de Valor Tributario) para el 2023.

In [5]:
#rangos en uvt
rango = np.arange(28, 1201)

#convertir rango a tipo float
rango = rango.astype(float)

#agregar el valor e de 27.35 como primer elemento
rango = np.insert(rango, 0, salario_minimo/uvt)

In [6]:
salario_col = pd.DataFrame(rango, columns=['n_uvt'])
salario_col['salario_pesos'] = ((salario_col['n_uvt'] * uvt)
                                .astype(int))

In [7]:
# funcion para calcular el pago de salud y pension
def pago_salud_pension(salario_pesos: int,
                       salario_integral: int = salario_integral
                       )-> int:
    """ 
    Calcula el pago de salud y pension para un salario dado en pesos,
    es igual al 4% del salario si es menor al salario integral,
    si es mayor al salario integral es igual al 4% del 70% del salario
    
    Args:
        salario_pesos (int): salario en pesos
        salario_integral (int): salario integral en pesos
    Returns:
        int: pago de salud y pension       
    
    """
    
    if salario_pesos < salario_integral:
        if salario_pesos < salario_minimo:
            aportes_salud_pension = salario_minimo * 0.04
        else:
            aportes_salud_pension =  salario_pesos * 0.04
        
    else:
        aportes_salud_pension = salario_pesos *.7 * 0.04
         
    return aportes_salud_pension

In [8]:
salario_col['pago_salud'] = (salario_col['salario_pesos']
                                   .apply(pago_salud_pension,
                                          salario_integral=salario_integral))
salario_col['pago_pension'] = salario_col['pago_salud']

In [9]:
# Auxilio de transporte igual a 140606 si salario es menor a 2 salarios minimos,
# de lo contrario es 0

salario_col['aux_transporte'] = np.where(salario_col['salario_pesos'] <= 2*salario_minimo,
                                            subsido_transporte, 0)



In [10]:
# funcion para calcular la retencion en la fuente

def retencion_fuente(n_uvt:float,
                     uvt:int )->float:
    """
    Calcula la retencion en la fuente para un salario en uvt
    
    Args:
        n_uvt (float): salario en uvt
        uvt (int): valor de un uvt
    
    Returns:
        float: valor de la retencion en la fuente en pesos       
    """
    # la tupla se compone de:
    # limite inferior, limite superior, porcentaje, uvt que se suman
    rangos = [(0,   95,  0, 0),
              (96,  150, 0.19, 0),
              (151, 360, 0.28, 10),
              (361, 640, 0.33, 69),
              (641, 945, 0.35, 162),
              (946, 2300, 0.37, 268)]
    retencion = 0
    for limite_inf, limite_sup, porcentaje, base in rangos:
        if (limite_inf <= n_uvt <= limite_sup):
            retencion = (n_uvt - (limite_inf-1)) * porcentaje + base
            break
    else:
        retencion = (n_uvt - 2300) * 0.37 + 268
    return retencion * uvt        

In [11]:
salario_col['retencion_fuente'] = (salario_col['n_uvt']
                                   .apply(retencion_fuente,
                                          uvt=uvt))
salario_col['deduccion_mensual'] = (salario_col['pago_salud']
                              + salario_col['pago_pension']
                              + salario_col['retencion_fuente'])

In [12]:
# Dinero entregado al empleado
salario_col['salario_neto_mes'] = (salario_col['salario_pesos']
                               + salario_col['aux_transporte']
                               - salario_col['deduccion_mensual'])

# salario integral en uvt
salario_inte_uvt = salario_integral/uvt

In [19]:
# grafiar salario_pesos, salario_neto vs n_uvt en plotly
fig = px.line(salario_col, x='n_uvt', y=['salario_pesos', 'salario_neto_mes'],
              title='Salario y Salario neto (Luego de las deducciones)'
                    ' [Mensual sin prima]',
              labels={'n_uvt': 'Salario en UVT',
                      'value': 'Salario en pesos',
                      'variable': 'Tipo de salario'})

#linea vertical en salario integral alpha=0.5
fig.add_vline(x=salario_inte_uvt, line_width=3, line_dash="dot",
              line_color="green", opacity=0.5,
              annotation_text="Salario Integral",
              annotation_position="bottom right")

#area entre n_uvt = 96 y n_uvt = 150
fig.add_vrect(x0=96, x1=150, line_width=0, fillcolor="cyan", opacity=0.2,
              annotation_text="Retencion 19%")

#area entre n_uvt = 151 y n_uvt = 360\
fig.add_vrect(x0=151, x1=360, line_width=0, fillcolor="yellow", opacity=0.2,
              annotation_text="Retencion 28%")
#area entre n_uvt = 361 y n_uvt = 640
fig.add_vrect(x0=361, x1=640, line_width=0, fillcolor="blue", opacity=0.2,
              annotation_text="Retencion 33%")
#area entre n_uvt = 641 y n_uvt = 945
fig.add_vrect(x0=641, x1=945, line_width=0, fillcolor="green", opacity=0.2,
              annotation_text="Retencion 35%")
#area entre n_uvt = 946 y n_uvt = 1200
fig.add_vrect(x0=946, x1=1200, line_width=0, fillcolor="purple", opacity=0.2,
              annotation_text="Retencion 37%")

fig.show()

grafica salario en pesos, salario recibido Anual

In [21]:
salario_col['prima'] = (salario_col['salario_pesos'] 
                        + salario_col['aux_transporte'])/2
salario_col['salario_con_prima'] = (salario_col['salario_pesos']
                                    + salario_col['prima'])

# retencion de salario mensual con prima
salario_col['retencion_fuente_prima'] = (salario_col['salario_con_prima']
                                            .apply(retencion_fuente,
                                            uvt=uvt))

# si salario_pesos es mayor de salario_integral prima es 0
salario_col['prima'] = np.where(salario_col['salario_pesos'] >= salario_integral,
                                0, salario_col['prima'])

# las cesantias son igual a un salario completo
salario_col['cesantias'] = salario_col['prima']*2

salario_col['deduccion_mensual_prima'] = (salario_col['pago_salud']
                                        + salario_col['pago_pension']
                                        + salario_col['retencion_fuente_prima'])



In [22]:
salario_col['salario_anual'] = (salario_col['salario_pesos'] * 12 
                                + salario_col['prima']*2 
                                + salario_col['cesantias'])

salario_col['deduccion_anual'] = (salario_col['deduccion_mensual'] * 10
                                  + salario_col['deduccion_mensual_prima']*2)


#falta hacer el salario_anual_neto tener en cuenta
# la retencion en la fuente de los meses que se recibe la prima
salario_col['salario_anual_neto'] = (salario_col['salario_anual']
                                     - salario_col['deduccion_anual'])

# Refencias

- https://blog.alegra.com/retencion-en-la-fuente/
- https://www.consultorcontable.com/retenci%C3%B3n-salarios/
- https://loggro.com/blog/articulo/conozca-como-quedo-el-salario-minimo-y-salario-integral-en-2023/
- https://loggro.com/blog/articulo/como-calcular-el-salario-integral-loggro-nomina/
- 

